# Seminario 3 de Python: Iteradores y generadores.

En este seminario vamos a estudiar varios mecanismos muy importantes y útiles de Python, así como los conceptos en que se basan: los *iterables*, *iteradores* y *generadores*.

El concepto de iterador proviene de los *lenguajes funcionales*, como también sucede con las *comprehensions*. Python, aunque por diseño es un lenguaje orientado a objetos, incorpora conceptos y herramientas que provienen de lenguajes imperativos (estructurados) y de los lenguajes funcionales, lo que realmente lo convierte en un lenguaje multiparadigma. Los iteradores son parte integral de Python y su utilización es ubicua, hasta el punto de que muchas veces no se perciben de forma evidente.


## Iterables y contextos de iteración.

Al inicio del aprendizaje de Python se introduce el concepto de *secuencia*, como son las cadenas, listas, tuplas, o archivos; se dice también que la función `range` crea una secuencia numérica; y se explica que la forma natural de recorrer una secuencia de inicio a fin es utilizar la sentencia `for` del siguiente modo:

    for variable in secuencia:
        ...

Sin embargo, se trata de una simplificación de un concepto mucho más general: el de **iterable**. Un objeto iterable es aquél que permite realizar una iteración sobre todos sus elementos. Las secuencias son obviamente iterables, pero hay muchos otros objetos que, no siendo secuencias, también son iterables.

Todo objeto iterable se puede utilizar en un **contexto de iteración**, como es el caso de la sentencia `for`. De modo que, en realidad, un `for` tiene esta forma:

    for variable in iterable:
        ...

En Python existen numerosos ejemplos de contextos de iteración. Por ejemplo, los desempaquetamientos, sean implícitos (como en una asignación múltiple) o explícitos (al utilizar los operadores `*` y `**`) definen contextos de iteración. También los encontramos (aunque de forma más evidente) en las *comprehensions* y en las expresiones generadoras.


## Iterables e iteradores.

Formalmente, un objeto es *iterable* si posee un método `__iter__`. Dicho método ha de retornar un objeto **iterador**, que se caracteriza por utilizar el denominado *protocolo de iterador* (*iterator protocol*), el cual consiste en:

- Poseer un método `__next__`, que produce y retorna un elemento a partir del iterable. Si no quedan más elementos, el iterador desencadena la excepción `StopIteration`.
- Poseer un método `__iter__` que retorna el mismo objeto iterador. Esto implica que todo iterador es también un iterable.

En definitiva: *el objeto iterador itera sobre los elementos del iterable del que procede*.

El método `__next__` del iterador producirá los elementos del iterable en un orden concreto: si el iterable es una secuencia, utilizará el orden de dicha secuencia; pero si el iterable no define un orden para sus elementos, no debemos realizar suposiciones sobre el orden de producción por parte de su iterador.

Por otra parte, un iterador producirá elementos a partir del iterable del que procede, pero *no tienen por qué ser exactamente los elementos del iterable*. Por ejemplo, los iteradores sobre diccionarios producen sólo las claves.

Python ofrece, como conveniencia, las funciones `iter` y `next`, que invocan respectivamente los métodos `__iter__` y `__next__` del objeto que se les pasa como argumento, y que retornan lo mismo que sendos métodos.


### Ejemplos.

Empecemos por definir una lista `l`. Como ya hemos dicho, las listas son objetos iterables; es por esa razón que las podemos recorrer dentro de contextos de iteración, como un `for`:

In [1]:
l = ['hola', 'qué', 'tal']
for c in l:
    print(c)

hola
qué
tal


Ahora veamos qué sucede en el `for` con más detalle. Lo primero que sucede en un contexto de iteración es que Python invoca el método `__iter__` del objeto iterable (en este ejemplo, la lista `l`). Nosotros lo haremos utilizando la función `iter` por simple comodidad:

In [2]:
it = iter(l)
print(it)

Obsérvese que `iter(l)` ha retornado un objeto iterador (de clase `list_iterator`) que itera sobre la lista `l`. Como todo iterador, este objeto cuenta con un método `__next__` que, al invocarlo, retornará un nuevo elemento de `l`. Dado que se trata de una lista, sus elementos tienen un orden definido, por lo que el iterador `it` los producirá en ese mismo orden.

A continuación invocaremos dicho método, utilizando la función `next`:

In [3]:
print(next(it))
print(next(it))
print(next(it))

hola
qué
tal


Nótese que el iterador `it` retiene el estado tras cada invocación de su método `__next__`: es decir, *recuerda* en qué punto de la secuencia ha quedado. (Nótese que esto puede provocar efectos inesperados si la célula anterior de este notebook se ejecuta más veces sin volver a ejecutar la célula previa.)

En este punto, hemos obtenido los tres elementos de la lista `l`. Veamos qué sucede al ejecutar `next(it)` una vez más:

In [4]:
print(next(it))

StopIteration: 

Como podemos comprobar, surge la excepción `StopIteration`. Aquí vemos el mensaje de error porque no se ha capturado esa excepción, pero en los contextos de iteración sí se captura, con el objeto de salir de dicho contexto cuando al iterador ya no le quedan elementos por producir. En una sentencia `for`, la salida del contexto de iteración consiste simplemente en finalizar el ciclo. De hecho, podemos simular un `for` como el de más arriba fácilmente:

> Nota: aunque el manejo de excepciones en Python se tratará en un seminario posterior, el uso que realizamos a continuación debería entenderse fácilmente.

In [5]:
it = iter(l)
while True:
    try:
        c = next(it)
    except StopIteration:
        break
    print(c)

hola
qué
tal


> Nota: si alguien está considerando  modificar la célula anterior para reemplazar `next(it)` por `next(iter(l))` y así eliminar la primera línea, animo a que lo pruebe, y encuentre la explicación a lo que sucede.

Un fenómeno que a primera vista puede parecer sorprendente es que el funcionamiento de un iterador puede verse afectado por posibles modificaciones al iterable del que procede. Esto sucede porque el objeto iterador utiliza los elementos del objeto iterable:

In [6]:
l = [1,2]
it = iter(l)
print(next(it))
print(next(it))
l.insert(0,0)
print(next(it))
l.append(3)
print(next(it))

1
2
2
3


Ahora bien, una vez que se ha agotado el iterador (habiéndose desencadenado `StopIteration`), su estado es irreversible. En cualquier caso, no es prudente modificar un iterable mientras se utiliza un iterador creado a partir de él.

Por otra parte, no hay problema en utilizar simultáneamente varios iteradores creados a partir del mismo iterable:

In [7]:
l = [1,2,3,4]
it1, it2 = iter(l), iter(l)
print('it1',next(it1))
print('it1',next(it1))
print('it2',next(it2))
print('it1',next(it1))
print('it2',next(it2))
print('it2',next(it2))
print('it1',next(it1))
print('it2',next(it2))

it1 1
it1 2
it2 1
it1 3
it2 2
it2 3
it1 4
it2 4


También podemos comprobar que *el método `__iter__` de un iterador retorna el mismo iterador*, lo que permite emplear un iterador allí donde se pueda utilizar un iterable:

In [8]:
print(id(it))
print(id(iter(it)))

140384570238432
140384570238432


### Iteradores sobre diccionarios.

Como ya sabemos, los objetos de clase `dict` cuentan con los métodos `keys`, `values` e `items`, los cuales retornan sendas *vistas* (*views*) sobre las claves, los valores y tuplas clave-valor, respectivamente. Dichas *vistas* son objetos iterables, y como su nombre indica funcionan como “ventanas”, por lo que no consumen memoria adicional. Por tanto, podemos iterar sobre un diccionario de forma muy sencilla:

In [9]:
d = {'a': 1, 'b': 2, 'c': 3}

for k in d.keys():
    print(k)
for v in d.values():
    print(v)
for i in d.items():
    print(i)

a
b
c
1
2
3
('a', 1)
('b', 2)
('c', 3)


Sucede, no obstante, que los objetos `dict` son en sí mismos iterables:

In [10]:
it = iter(d)
print(next(it))
print(next(it))
print(next(it))

a
b
c


Como podemos ver, el método `__next__` de un iterador no está obligado a producir elementos del iterable: puede diseñarse para que retorne lo que se considere más apropiado. Así, los iteradores sobre diccionarios producen sus claves (desde Python 3.7, el orden de las claves de un diccionario es el de su inserción). De modo que si simplemente deseamos iterar sobre las claves de un diccionario, podemos prescindir del método `keys`:

In [11]:
for k in d:
    print(k)

a
b
c


### Otros objetos iterables.

No sólo los objetos de las clases contenedoras son iterables; existen muchos tipos de objetos que también poseen el método `__iter__`. Por ejemplo, la función `range` crea y retorna un objeto iterable de clase `range` que, en realidad, no contiene una estructura de datos como una lista o tupla con todos los elementos del rango, sino *la forma de producir esos elementos*. Lógicamente, un iterador creado a partir de un objeto `range` produce secuencialmente cada elemento del rango cuando se invoca su método `__next__`.

In [12]:
r = range(10)
it = iter(r)
print('r:', r)
print('iter(r):', it)
print(next(it), next(it), next(it))

r: range(0, 10)
iter(r): <range_iterator object at 0x7fadd470e960>
0 1 2


También existen funciones que crean *directamente* objetos iteradores que, por tanto, no proceden de un iterable. Por ejemplo, la función `zip` sirve para emparejar (o “n-tuplar”, en realidad) los elementos de dos o más secuencias… o iterables. Pues bien, esta función *retorna directamente un iterador*; obsérvese a continuación que no es necesario utilizar el método `__iter__` para crear un iterador a partir del objeto que retorna `zip`:

In [13]:
z = zip([1,2,3],[4,5,6])
print(next(z), next(z), next(z))

(1, 4) (2, 5) (3, 6)


Recuérdese que se puede utilizar un iterador allí donde se pueda emplear un iterable, por lo que podemos utilizar `zip` en un `for`:

In [14]:
for t in zip(range(0,5),range(1,6),range(2,7)):
    print(t)

(0, 1, 2)
(1, 2, 3)
(2, 3, 4)
(3, 4, 5)
(4, 5, 6)


Obsérvese que, de forma similar a `range`, el iterador construido por la función `zip` no contiene todos los elementos, sino la forma de producirlos. En este último ejemplo en el que se le pasan como argumentos tres objetos `range`, el uso de memoria es mínimo, ya que ni los objetos `range` (ni los iteradores sobre ellos), ni el objeto `zip` contienen elemento alguno (enteros o tuplas), sino solamente el código necesario para producirlos en el momento en que se invoque el método `__next__`. En consecuencia, podemos afirmar que **el uso de iteradores resulta extremadamente eficiente en cuanto a consumo de memoria**.


## Definición de iteradores.

Como ya hemos visto, un iterador básicamente genera una sucesión de elementos, produciendo y retornando un nuevo elemento al invocar su método `__next__`; si no hay más elementos, desencadena la excepción `StopIteration`. También posee un método `__iter__` que simplemente retorna el mismo objeto iterador.

Nada nos impide crear nuestros propios iteradores, definiendo una clase cuyas instancias cumplan con los requisitos mencionados. Como ejemplo, a continuación se muestra la definición de la clase `Fiboserie`. Un iterador de clase `Fiboserie` genera una secuencia correspondiente a la serie de Fibonacci. Al constructor de esta clase se le pasa como argumento de elementos deseados.

In [15]:
class Fiboserie():
    def __init__(self, nterms):
        self._nterms = nterms
        self._n = 0
        self._a, self._b = 0, 1
    def __next__(self):
        if self._n == self._nterms:
            raise StopIteration
        else:
            self._n += 1
            self._a, self._b = self._b, self._a + self._b
            return self._a
    def __iter__(self):
        return self

Como es habitual, el método `__init__` inicializa cada instancia que se crea de `Fiboserie`. Posee un parámetro `nterms` que, obviamente, se utiliza para especificar el número máximo de términos de la serie de Fibonacci que se han de generar. El valor de este parámetro se guarda en una variable privada de la instancia denominada  `_nterms`. También se definen las variables privadas `_n` (que lleva la cuenta del número de términos ya generados), así como `_a` y `_b` donde se mantienen los dos últimos términos calculados de la serie.

> La convención en Python es que los nombres de los atributos “*privados*” de una clase comiencen con un guión bajo (`_`), como forma de indicar que no está previsto que se acceda externamente a esos atributos (en realidad, nada lo impide). Si el nombre de un atributo comienza con dos guiones bajos (`__`) Python lo altera para que no entre en conflicto con atributos de posibles subclases.

El método `__next__` comienza por comprobar si ya se ha generado el número máximo de términos de la serie, en cuyo caso desencadena `StopIteration`. Si no es así, incrementa `_n` y avanza `_a` y `_b` a los siguientes términos respectivos. Finalmente, retorna `_a`.

El método `__iter__` se limita a que esa instancia del iterador se retorne a sí misma.

Nótese que las variables `_nterms`, `_n`, `_a` y `_b` son propias de cada instancia de `Fiboserie`, por lo que no hay problema alguno en que existan simultáneamente varios iteradores `Fiboserie`, dado que no hay interferencias mutuas en su funcionamiento. Veamos unos ejemplos de uso:

In [16]:
for n in Fiboserie(10):
    print(n)

1
1
2
3
5
8
13
21
34
55


In [17]:
l = list(Fiboserie(20))
print('l =', l)

l = [1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765]


El cociente entre un término de la serie de Fibonacci y el anterior tiene como límite la *razón aúrea* (𝜑). Supongamos que queremos (por puro masoquismo) calcular 𝜑 de ese modo, en lugar utilizar directamente la fórmula 𝜑=(1+√5)/2. La cuestión es cómo conseguir *sólo* los dos últimos términos generados por `Fibogen(50)` (con 50 términos ya se consigue muy buena precisión) sin almacenarlos previamente en una lista u otra estructura, ya que eso consumiría memoria sin necesidad.

In [18]:
# Cálculo directo con la conocida fórmula:
print('Usando fórmula: 𝜑 =', (1+5**0.5)/2)

# Consume memoria para la lista:
a,b = list(Fiboserie(50))[-2:]
print('Usando lista: 𝜑 =',b/a)

# Consume memoria, pero queda elegante:
*borrame,a,b = Fiboserie(50); del borrame
print('Usando empaquetamiento: 𝜑 =',b/a)

# Hay muchas más formas que de una manera u otra
#  almacenan los términos que no necesitamos...

# Pero así no se consume más memoria de la necesaria:
fibo_it = Fiboserie(50)
for a in fibo_it:
    b = next(fibo_it)
print('Usando for con ‘next’ extra: 𝜑 =', b/a)

Usando fórmula: 𝜑 = 1.618033988749895
Usando lista: 𝜑 = 1.618033988749895
Usando empaquetamiento: 𝜑 = 1.618033988749895
Usando for con ‘next’ extra: 𝜑 = 1.618033988749895


Nótese que el *truco* del último método sólo funciona con un número par de términos. Si fuese impar, el `next` dentro del ciclo desencadenaría `StopIteration` justo al final. Podríamos capturar la excepción, eso sí, pero nos quedaría en `a` el último término y en `b` el penúltimo, por lo que habría que hacer `a/b`.


## Generadores.

Los generadores son una herramienta que posibilita crear iteradores de forma sencilla y flexible. En su forma más general, se parecen a funciones convencionales, salvo que utilizan la sentencia `yield` para retornar datos. La diferencia entre las sentencias `yield` y `return` es que la primera hace que el generador conserve su estado, de forma que la próxima vez que se invoca `__next__` continúa en el punto en que se quedó tras ejecutarse `yield`, recordando también todas sus variables locales.

Veamos un ejemplo de generador que funciona de manera similar al iterador `Fiboserie`:

In [19]:
def Fibogen(nterms):
    n = 0
    a,b = 0,1
    
    while n < nterms:
        n += 1
        a,b = b,a+b
        yield a

Como podemos comprobar, la definición de un generador resulta simple y compacta, ya que los métodos `__next__` e `__iter__` se crean automáticamente, y la excepción `StopIteration` se desencadena al alcanzarse el final de su “función”. Por supuesto, el generador `Fibogen` funciona y se utiliza exactamente igual que el iterador `Fiboserie`:

In [20]:
for n in Fibogen(10):
    print(n)

1
1
2
3
5
8
13
21
34
55


### Expresiones generadoras.

Algunos generadores sencillos se pueden escribir de manera aún más compacta en forma de *expresiones generadoras*. Su sintaxis es prácticamente idéntica a la de las *list comprehensions*, pero utilizando paréntesis en lugar de corchetes. (Los paréntesis no son necesarios si la expresión generadora es el único argumento de una función.)

Las expresiones generadoras se pueden utilizar en lugar de *list comprehensions* cuando no necesitamos crear una lista como tal, sino tan sólo iterar sobre sus elementos. Por ejemplo, compárese

In [21]:
s1 = sum(n*n for n in range(10))
s2 = sum([n*n for n in range(10)])

El efecto es exactamente el mismo, ya que la función `sum` espera realmente un *iterable* como argumento. La diferencia es que con la expresión generadora no se construye lista alguna, por lo que resulta más eficiente en tiempo y en uso de memoria.

En el siguiente ejemplo se calcula un producto escalar de dos vectores (listas) sin necesidad de construir ninguna lista adicional:

In [22]:
vec1 = [10, 20, 30, 40, 50]
vec2 = [2, 3, 5, 7, 11]
print(sum(x*y for x,y in zip(vec1,vec2)))

1060


La función `sum` utiliza el generador que se le ha pasado como argumento para ir obteniendo los sumandos uno por uno, que va acumulando en el sumatorio. A su vez, cada vez que se le demanda un nuevo elemento desde la función, el generador obtiene del iterador creado por `zip` una nueva tupla, de la cual extrae y multiplica sus dos componentes; y a su vez, el iterador del `zip` obtiene un par de elementos de ambos vectores y forma con ellos una tupla a medida que se le va demandando por parte del generador. Éste es un proceso muy eficiente en términos de uso de memoria, y también temporales.

Quizá se vea esto más claramente, y se aprecie mejor la potencia y facilidad de uso de los iteradores, generadores y expresiones generadoras si descomponemos la expresión anterior del producto escalar de la siguiente forma (`mi_sum` y `mi_zip` son simplificaciones de las funciones `sum` y `zip`):

In [23]:
def mi_sum(iterable):
    suma = 0
    for item in iterable:
        suma += item
    return suma
    
def prod_elems(v1,v2):
    for x,y in mi_zip(v1,v2):
        yield x*y
    
def mi_zip(iterable1, iterable2):
    it1 = iter(iterable1)
    it2 = iter(iterable2)
    while True:
        try:
            yield next(it1), next(it2)
        except StopIteration:
            break

print(mi_sum(prod_elems(vec1,vec2)))

1060


## El módulo `itertools`.

La biblioteca estándar de Python cuenta con un módulo llamado `itertools`, el cual ofrece un conjunto de iteradores que, en conjunto, forman un “álgebra de iteradores“ con los que se pueden realizar construcciones compactas y extremadamente potentes; más aún si se combinan con el módulo `operator` de la biblioteca estándar.

No vamos a tratar aquí el contenido y uso del módulo `itertools`, pero recomendamos [la lectura de su documentación](https://docs.python.org/3/library/itertools.html), que además incluye diversos ejemplos de uso.